In [1]:
import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
all_data = [train_data, test_data]

In [3]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Feature engineering

### Passenger class

In [4]:
print( train_data[["Pclass","Survived"]].groupby(["Pclass"], as_index = False).mean() )

   Pclass  Survived
0       1  0.629630
1       2  0.472826
2       3  0.242363


### Sex

In [5]:
print( train_data[["Sex", "Survived"]].groupby(["Sex"], as_index = False).mean())

      Sex  Survived
0  female  0.742038
1    male  0.188908


### Family size

In [6]:
for data in all_data:
    data['family_size'] = data['SibSp'] + data['Parch'] + 1
print( train_data[["family_size", "Survived"]].groupby(["family_size"], as_index = False).mean())

   family_size  Survived
0            1  0.303538
1            2  0.552795
2            3  0.578431
3            4  0.724138
4            5  0.200000
5            6  0.136364
6            7  0.333333
7            8  0.000000
8           11  0.000000


Let's check if the person was alone on the board

In [7]:
for data in all_data:
    data['is_alone'] = 0
    data.loc[data['family_size'] == 1, 'is_alone'] = 1
print( train_data[["is_alone", "Survived"]].groupby(["is_alone"], as_index = False).mean())

   is_alone  Survived
0         0  0.505650
1         1  0.303538


### Embarked

In [8]:
for data in all_data:
    data['Embarked'] = data['Embarked'].fillna('S')
print(train_data[['Embarked', 'Survived']].groupby(['Embarked'], as_index = False).mean())

  Embarked  Survived
0        C  0.553571
1        Q  0.389610
2        S  0.339009


### Fare

In [9]:
for data in all_data:
    data['Fare'] = data['Fare'].fillna(data['Fare'].median())
train_data['category_fare'] = pd.qcut(train_data['Fare'], 4)
print(train_data[['category_fare', 'Survived']].groupby(['category_fare'], as_index = False).mean())

     category_fare  Survived
0   (-0.001, 7.91]  0.197309
1   (7.91, 14.454]  0.303571
2   (14.454, 31.0]  0.454955
3  (31.0, 512.329]  0.581081


### Age

In [10]:
for data in all_data:
    age_avg = data['Age'].mean()
    age_std = data['Age'].std()
    age_null = data['Age'].isnull().sum()
    
    random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size = age_null)
    data['Age'][np.isnan(data['Age'])] = random_list
    data['Age'] = data['Age'].astype(int)

train_data['category_age'] = pd.cut(train_data['Age'], 5)
print(train_data[['category_age', 'Survived']].groupby(['category_age'], as_index = False).mean())

    category_age  Survived
0  (-0.08, 16.0]  0.535714
1   (16.0, 32.0]  0.356044
2   (32.0, 48.0]  0.364754
3   (48.0, 64.0]  0.434783
4   (64.0, 80.0]  0.090909


/Users/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


### Name
Use regular expressions to retrieve person's title: Mr, Sr, Captain

In [11]:
import re

def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\. ', name)
    if title_search:
        return title_search.group(1)
    return ""

for data in all_data:
    data['title'] = data['Name'].apply(get_title)

for data in all_data:
    data['title'] = data['title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'],'Rare')
    data['title'] = data['title'].replace('Mlle', 'Miss')
    data['title'] = data['title'].replace('Ms', 'Miss')
    data['title'] = data['title'].replace('Mme', 'Mrs')

print(pd.crosstab(train_data['title'], train_data['Sex']))
print('----------------------')
print(train_data[['title', 'Survived']].groupby(['title'], as_index = False).mean())

Sex     female  male
title               
Master       0    40
Miss       185     0
Mr           0   517
Mrs        126     0
Rare         3    20
----------------------
    title  Survived
0  Master  0.575000
1    Miss  0.702703
2      Mr  0.156673
3     Mrs  0.793651
4    Rare  0.347826


### Mapping data

In [12]:
#Map Data
for data in all_data:

    #Mapping Sex
    sex_map = { 'female':0 , 'male':1 }
    data['Sex'] = data['Sex'].map(sex_map).astype(int)

    #Mapping Title
    title_map = {'Mr':1, 'Miss':2, 'Mrs':3, 'Master':4, 'Rare':5}
    data['title'] = data['title'].map(title_map)
    data['title'] = data['title'].fillna(0)

    #Mapping Embarked
    embark_map = {'S':0, 'C':1, 'Q':2}
    data['Embarked'] = data['Embarked'].map(embark_map).astype(int)

    #Mapping Fare
    data.loc[ data['Fare'] <= 7.91, 'Fare']                            = 0
    data.loc[(data['Fare'] > 7.91) & (data['Fare'] <= 14.454), 'Fare'] = 1
    data.loc[(data['Fare'] > 14.454) & (data['Fare'] <= 31), 'Fare']   = 2
    data.loc[ data['Fare'] > 31, 'Fare']                               = 3
    data['Fare'] = data['Fare'].astype(int)

    #Mapping Age
    data.loc[ data['Age'] <= 16, 'Age']                       = 0
    data.loc[(data['Age'] > 16) & (data['Age'] <= 32), 'Age'] = 1
    data.loc[(data['Age'] > 32) & (data['Age'] <= 48), 'Age'] = 2
    data.loc[(data['Age'] > 48) & (data['Age'] <= 64), 'Age'] = 3
    data.loc[ data['Age'] > 64, 'Age']                        = 4

#Feature Selection
#Create list of columns to drop
drop_elements = ["Name", "Ticket", "Cabin", "SibSp", "Parch", "family_size"]

#Drop columns from both data sets
train_data = train_data.drop(drop_elements, axis = 1)
train_data = train_data.drop(['PassengerId','category_fare', 'category_age'], axis = 1)
test_data = test_data.drop(drop_elements, axis = 1)

#Print ready to use data
print(train_data.head(10))

   Survived  Pclass  Sex  Age  Fare  Embarked  is_alone  title
0         0       3    1    1     0         0         0      1
1         1       1    0    2     3         1         0      3
2         1       3    0    1     1         0         1      2
3         1       1    0    2     3         0         0      3
4         0       3    1    2     1         0         1      1
5         0       3    1    2     1         2         1      1
6         0       1    1    3     3         0         1      1
7         0       3    1    0     2         0         0      4
8         1       3    0    1     1         0         0      3
9         1       2    0    0     2         1         0      3


In [13]:
train_data.dtypes

Survived    int64
Pclass      int64
Sex         int64
Age         int64
Fare        int64
Embarked    int64
is_alone    int64
title       int64
dtype: object

### Dealing with dummy variables
Let's convert Pclass, Age, Fare, Embarked and title to categorical variable

In [14]:
categorical = ["Pclass", "Age", "Fare", "Embarked", "title"]
for x in categorical:
    train_data[x] = train_data[x].apply(str)
    test_data[x] = test_data[x].apply(str)

In [15]:
train_data.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,is_alone,title
0,0,3,1,1,0,0,0,1
1,1,1,0,2,3,1,0,3
2,1,3,0,1,1,0,1,2
3,1,1,0,2,3,0,0,3
4,0,3,1,2,1,0,1,1


In [16]:
train_data = pd.get_dummies(train_data)
test_data = pd.get_dummies(test_data)
drop_columns = ['Pclass_1', 'Age_0', 'Fare_1', 'Embarked_0']
train_data = train_data.drop(drop_columns,axis = 1)
test_data = test_data.drop(drop_columns, axis = 1)

## Prediction

In [17]:
X_train = train_data.drop("Survived", axis=1)
Y_train = train_data["Survived"]
X_test  = test_data.drop("PassengerId", axis=1).copy()

### normalize 

In [18]:
from sklearn import preprocessing
X_train = preprocessing.normalize(X_train)
X_test = preprocessing.normalize(X_test)

In [19]:
X_train

array([[0.4472136 , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.5       , 0.        ,
        0.        ],
       [0.        , 0.5       , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.40824829, 0.40824829, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.37796447, 0.37796447, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [20]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression().fit(X_train,Y_train)

/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [21]:
Y_pred = model.predict(X_test)

In [22]:
pass_id = pd.read_csv('test.csv')['PassengerId']
pass_id

0       892
1       893
2       894
3       895
4       896
5       897
6       898
7       899
8       900
9       901
10      902
11      903
12      904
13      905
14      906
15      907
16      908
17      909
18      910
19      911
20      912
21      913
22      914
23      915
24      916
25      917
26      918
27      919
28      920
29      921
       ... 
388    1280
389    1281
390    1282
391    1283
392    1284
393    1285
394    1286
395    1287
396    1288
397    1289
398    1290
399    1291
400    1292
401    1293
402    1294
403    1295
404    1296
405    1297
406    1298
407    1299
408    1300
409    1301
410    1302
411    1303
412    1304
413    1305
414    1306
415    1307
416    1308
417    1309
Name: PassengerId, Length: 418, dtype: int64

In [23]:
predictions = pd.DataFrame(data = list(zip(pass_id, Y_pred)), columns = ['PassengerId', 'Survived'])

In [24]:
predictions.to_csv('predictions.csv',index=False)